In [ ]:
#Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('/content/creditcard.csv')

In [ ]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
#Define x and y
X = df.drop('Class', axis=1)
y = df['Class']

In [ ]:
#Train-Test Split
X = df.drop(['Class'], axis=1)
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [ ]:
#Standardize Numeric Features (like ‘Amount’)
scaler = StandardScaler()
X_train[['Amount']] = scaler.fit_transform(X_train[['Amount']])
X_test[['Amount']] = scaler.transform(X_test[['Amount']])

In [ ]:
#Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

In [ ]:
#LightGBM
lgbm = LGBMClassifier(n_estimators=100, random_state=42)
lgbm.fit(X_train, y_train)
y_pred_lgbm = lgbm.predict(X_test)

[LightGBM] [Info] Number of positive: 114, number of negative: 36402
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012112 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7650
[LightGBM] [Info] Number of data points in the train set: 36516, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.003122 -> initscore=-5.766181
[LightGBM] [Info] Start training from score -5.766181
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

In [ ]:
#Function for evaluating each model
def evaluate_model(name, y_true, y_pred):
    print(f"--- {name} ---")
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
    print("Classification Report:\n", classification_report(y_true, y_pred))
    print("ROC AUC:", roc_auc_score(y_true, y_pred))
    print("\n")

In [ ]:
evaluate_model("Logistic Regression", y_test, y_pred_lr)
evaluate_model("Random Forest", y_test, y_pred_rf)
evaluate_model("LightGBM", y_test, y_pred_lgbm)

--- Logistic Regression ---
Confusion Matrix:
 [[9089   12]
 [   8   20]]
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      9101
         1.0       0.62      0.71      0.67        28

    accuracy                           1.00      9129
   macro avg       0.81      0.86      0.83      9129
weighted avg       1.00      1.00      1.00      9129

ROC AUC: 0.8564835889305729


--- Random Forest ---
Confusion Matrix:
 [[9099    2]
 [   2   26]]
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      9101
         1.0       0.93      0.93      0.93        28

    accuracy                           1.00      9129
   macro avg       0.96      0.96      0.96      9129
weighted avg       1.00      1.00      1.00      9129

ROC AUC: 0.9641758362503335


--- LightGBM ---
Confusion Matrix:
 [[9099    2]
 [   5   23]]
Classification Report:
          

In [ ]:
#Save Random forest for deployment
import joblib
final_model = rf  # RandomForestClassifier
joblib.dump(final_model, 'fraud_model_rf.pkl')

['fraud_model_rf.pkl']